<a href="https://colab.research.google.com/github/Warspyt/PC_Python_2025II/blob/main/clase__20/cajero_pyqt5_simulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🏧 Simulador de Cajero Automático (PyQt5)

Este notebook contiene instrucciones paso a paso y código listo para copiar y ejecutar localmente. PyQt5 **no** funciona en Colab; sigue las instrucciones para ejecutar en tu equipo.

## Requisitos

- Python 3.7+
- PyQt5: `pip install PyQt5`
- Ejecutar en entorno con servidor gráfico (Windows/macOS/Linux)


## Estructura recomendada

Crea carpeta `cajero_pyqt` con:
- `bank_logic.py`
- `ui_main.py`
- `cajero_app.py`


## 1) bank_logic.py

Código:

In [ ]:
# bank_logic.py
from datetime import datetime

class Account:
    def __init__(self, account_number, pin, balance=0.0):
        self.account_number = account_number
        self.pin = str(pin)
        self.balance = float(balance)
        self.history = []

    def check_pin(self, pin):
        return str(pin) == self.pin

    def deposit(self, amount):
        if amount <= 0:
            raise ValueError("El monto de depósito debe ser positivo.")
        self.balance += amount
        self._add_history('DEPÓSITO', amount)
        return self.balance

    def withdraw(self, amount):
        if amount <= 0:
            raise ValueError("El monto de retiro debe ser positivo.")
        if amount > self.balance:
            raise ValueError("Fondos insuficientes.")
        self.balance -= amount
        self._add_history('RETIRO', amount)
        return self.balance

    def _add_history(self, typ, amount):
        self.history.append((datetime.now().isoformat(timespec='seconds'), typ, float(amount), float(self.balance)))

    def get_balance(self):
        return self.balance

    def get_history(self, last_n=None):
        return list(self.history) if last_n is None else self.history[-last_n:]

class Bank:
    def __init__(self):
        self.accounts = {
            '123456': Account('123456', '1234', 1000.0),
            '654321': Account('654321', '4321', 250.0),
        }

    def find_account(self, account_number):
        return self.accounts.get(str(account_number))


## 2) ui_main.py

Código:

In [ ]:
# ui_main.py
import sys
from PyQt5.QtWidgets import (QApplication, QWidget, QLabel, QLineEdit, QPushButton,
                             QVBoxLayout, QInputDialog, QListWidget, QMainWindow, QAction, QFileDialog, QMessageBox)
from PyQt5.QtCore import Qt
from bank_logic import Bank

class LoginWindow(QWidget):
    def __init__(self, bank, on_login):
        super().__init__()
        self.bank = bank
        self.on_login = on_login
        self.setWindowTitle('Cajero - Login')
        self.setup_ui()

    def setup_ui(self):
        layout = QVBoxLayout()
        self.acc_input = QLineEdit(); self.acc_input.setPlaceholderText('Número de cuenta')
        self.pin_input = QLineEdit(); self.pin_input.setPlaceholderText('PIN'); self.pin_input.setEchoMode(QLineEdit.Password)
        login_btn = QPushButton('Ingresar')
        login_btn.clicked.connect(self.try_login)
        layout.addWidget(QLabel('Cuenta:')); layout.addWidget(self.acc_input)
        layout.addWidget(QLabel('PIN:')); layout.addWidget(self.pin_input)
        layout.addWidget(login_btn)
        self.setLayout(layout)

    def try_login(self):
        acc = self.acc_input.text().strip()
        pin = self.pin_input.text().strip()
        account = self.bank.find_account(acc)
        if not account:
            QMessageBox.warning(self, 'Error', 'Cuenta no encontrada.')
            return
        if not account.check_pin(pin):
            QMessageBox.warning(self, 'Error', 'PIN incorrecto.')
            return
        self.on_login(account)
        self.close()

class MainWindow(QMainWindow):
    def __init__(self, account):
        super().__init__()
        self.account = account
        self.setWindowTitle(f'Cajero - Cuenta {self.account.account_number}')
        self.setup_ui()

    def setup_ui(self):
        central = QWidget()
        layout = QVBoxLayout()

        self.balance_label = QLabel(f'Saldo: ${self.account.get_balance():.2f}')
        self.balance_label.setAlignment(Qt.AlignCenter)
        layout.addWidget(self.balance_label)

        btn_balance = QPushButton('Consultar Saldo')
        btn_withdraw = QPushButton('Retirar')
        btn_deposit = QPushButton('Depositar')
        btn_history = QPushButton('Historial')
        btn_exit = QPushButton('Salir')

        btn_balance.clicked.connect(self.show_balance)
        btn_withdraw.clicked.connect(self.withdraw)
        btn_deposit.clicked.connect(self.deposit)
        btn_history.clicked.connect(self.show_history)
        btn_exit.clicked.connect(self.close)

        for w in [btn_balance, btn_withdraw, btn_deposit, btn_history, btn_exit]:
            layout.addWidget(w)

        central.setLayout(layout)
        self.setCentralWidget(central)

        menubar = self.menuBar()
        file_menu = menubar.addMenu('Archivo')
        save_action = QAction('Guardar historial', self)
        save_action.triggered.connect(self.save_history)
        file_menu.addAction(save_action)

    def refresh_balance(self):
        self.balance_label.setText(f'Saldo: ${self.account.get_balance():.2f}')

    def show_balance(self):
        QMessageBox.information(self, 'Saldo', f'Su saldo es: ${self.account.get_balance():.2f}')

    def withdraw(self):
        amt, ok = QInputDialog.getDouble(self, 'Retirar', 'Monto a retirar:', decimals=2, min=0.01)
        if ok:
            try:
                self.account.withdraw(amt)
                QMessageBox.information(self, 'Éxito', f'Retiró ${amt:.2f}')
                self.refresh_balance()
            except Exception as e:
                QMessageBox.warning(self, 'Error', str(e))

    def deposit(self):
        amt, ok = QInputDialog.getDouble(self, 'Depositar', 'Monto a depositar:', decimals=2, min=0.01)
        if ok:
            try:
                self.account.deposit(amt)
                QMessageBox.information(self, 'Éxito', f'Depositó ${amt:.2f}')
                self.refresh_balance()
            except Exception as e:
                QMessageBox.warning(self, 'Error', str(e))

    def show_history(self):
        hist = self.account.get_history()
        dlg = QWidget()
        dlg.setWindowTitle('Historial de transacciones')
        v = QVBoxLayout()
        lst = QListWidget()
        for item in hist:
            ts, typ, amt, bal = item
            lst.addItem(f'{ts} | {typ} | ${amt:.2f} | Saldo: ${bal:.2f}')
        v.addWidget(lst)
        dlg.setLayout(v)
        dlg.resize(600,400)
        dlg.show()
        self._hist_dlg = dlg

    def save_history(self):
        import json
        fname, _ = QFileDialog.getSaveFileName(self, 'Guardar historial', '', 'JSON Files (*.json);;All Files (*)')
        if fname:
            with open(fname, 'w', encoding='utf-8') as f:
                json.dump(self.account.get_history(), f, ensure_ascii=False, indent=2)
            QMessageBox.information(self, 'Guardado', f'Historial guardado en {fname}')


## 3) cajero_app.py

Código:

In [ ]:
# cajero_app.py
import sys
from PyQt5.QtWidgets import QApplication
from bank_logic import Bank
from ui_main import LoginWindow, MainWindow

def main():
    app = QApplication(sys.argv)
    bank = Bank()
    main_win = None

    def on_login(account):
        nonlocal main_win
        main_win = MainWindow(account)
        main_win.show()

    login = LoginWindow(bank, on_login)
    login.show()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()


## 4) Ejecución local

1. Crea carpeta `cajero_pyqt` y coloca los 3 archivos (`bank_logic.py`, `ui_main.py`, `cajero_app.py`).
2. Instala PyQt5: `pip install PyQt5`.
3. Ejecuta: `python cajero_app.py`.

Cuentas de ejemplo:
- 123456 / PIN 1234 (saldo $1000.00)
- 654321 / PIN 4321 (saldo $250.00)


## 5) Extensiones sugeridas

- Persistencia con SQLite
- Límites diarios de retiro
- Impresión de recibos (PDF)
- Soporte multi-idioma
